<a href="https://colab.research.google.com/github/Otsemeuno/uk-road-accident/blob/main/projectone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **UK ROAD SAFET: Traffic Accidents and Vehicles**

In [ ]:
!pip install python-dotenv

## **Importing Libraries**

In [ ]:
from geopy.geocoders import Nominatim
from sklearn.impute import KNNImputer
import os
from dotenv import load_dotenv, find_dotenv
import requests
from math import radians, sin, cos, sqrt, atan2

In [ ]:
#importing data from google drive
from google.colab import drive
import pandas as pd
import numpy as np


## **Importing Data**

In [ ]:
# Mount Google Drive on Kernel
drive.mount('/content/drive')

#importing collision data
df_collision = pd.read_csv('/content/drive/MyDrive/dft-road-casualty-statistics-collision-last-5-years.csv')

#importing casaulty data
df_casaulty = pd.read_csv('/content/drive/MyDrive/dft-road-casualty-statistics-casualty-last-5-years.csv')

#importing vehicle data
df_vehicle = pd.read_csv('/content/drive/MyDrive/dft-road-casualty-statistics-vehicle-last-5-years.csv')

#importing district information
df_district = pd.read_csv('/content/drive/MyDrive/projectonedistrict - Sheet1.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-77-7397e153d0df>:5: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_collision = pd.read_csv('/content/drive/MyDrive/dft-road-casualty-statistics-collision-last-5-years.csv')
<ipython-input-77-7397e153d0df>:8: DtypeWarning: Columns (0,2,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_casaulty = pd.read_csv('/content/drive/MyDrive/dft-road-casualty-statistics-casualty-last-5-years.csv')
<ipython-input-77-7397e153d0df>:11: DtypeWarning: Columns (0,2,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vehicle = pd.read_csv('/content/drive/MyDrive/dft-road-casualty-statistics-vehicle-last-5-years.csv')


In [ ]:
# checking the file upload
df_collision.head()

,accident_index,accident_year,accident_reference,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,...,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location
0,2018010080971,2018,10080971,529150.0,182270.0,-0.139737,51.524587,1,3,2,...,0,4,1,1,0,0,1,1,2,E01000854
1,2018010080973,2018,10080973,542020.0,184290.0,0.046471,51.539651,1,3,1,...,0,4,1,1,0,0,1,1,2,E01003531
2,2018010080974,2018,10080974,531720.0,182910.0,-0.102474,51.529746,1,3,2,...,5,4,1,1,0,0,1,1,2,E01002723
3,2018010080981,2018,10080981,541450.0,183220.0,0.037828,51.530179,1,2,2,...,5,4,2,2,0,0,1,1,2,E01003492
4,2018010080982,2018,10080982,543580.0,176500.0,0.065781,51.469258,1,2,2,...,0,4,1,2,0,0,1,1,2,E01001682


In [ ]:
df_collision.shape

(538461, 36)

In [ ]:
df_casaulty.head()

,accident_index,accident_year,accident_reference,vehicle_reference,casualty_reference,casualty_class,sex_of_casualty,age_of_casualty,age_band_of_casualty,casualty_severity,pedestrian_location,pedestrian_movement,car_passenger,bus_or_coach_passenger,pedestrian_road_maintenance_worker,casualty_type,casualty_home_area_type,casualty_imd_decile,lsoa_of_casualty
0,2018010080971,2018,010080971,1,1,2,2,50,8,3,0,0,2,0,0,9,1,8,E01011051
1,2018010080971,2018,010080971,2,2,1,1,48,8,3,0,0,0,0,0,8,1,1,E01002066
2,2018010080973,2018,010080973,1,1,3,1,29,6,3,5,1,0,0,2,0,1,3,E01003531
3,2018010080974,2018,010080974,1,1,1,1,40,7,3,0,0,0,0,0,8,1,3,E01000185
4,2018010080981,2018,010080981,1,1,1,1,27,6,2,0,0,0,0,0,9,1,7,E01003643


In [ ]:
df_vehicle.head()

,accident_index,accident_year,accident_reference,vehicle_reference,vehicle_type,towing_and_articulation,vehicle_manoeuvre,vehicle_direction_from,vehicle_direction_to,vehicle_location_restricted_lane,...,sex_of_driver,age_of_driver,age_band_of_driver,engine_capacity_cc,propulsion_code,age_of_vehicle,generic_make_model,driver_imd_decile,driver_home_area_type,lsoa_of_driver
0,2018010080971,2018,10080971,1,9,0,18,3,7,0,...,1,32,6,1995,2,5,BMW 5 SERIES,8,1,E01011051
1,2018010080971,2018,10080971,2,8,0,18,3,7,0,...,1,48,8,1798,8,6,TOYOTA PRIUS+,1,1,E01002066
2,2018010080973,2018,10080973,1,9,0,18,3,7,0,...,3,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,2018010080974,2018,10080974,1,8,0,7,1,4,0,...,1,40,7,1797,8,6,TOYOTA PRIUS,3,1,E01000185
4,2018010080974,2018,10080974,2,9,0,18,4,8,0,...,1,21,5,-1,-1,-1,-1,5,1,E01000036


In [ ]:
df_district.head()

,Index,County
0,1,Metropolitan Police
1,3,Cumbria
2,4,Lancashire
3,5,Merseyside
4,6,Greater Manchester


##**Data Preparation**

### Variable Identification

In [ ]:
df_collision.columns

Index(['accident_index', 'accident_year', 'accident_reference',
       'location_easting_osgr', 'location_northing_osgr', 'longitude',
       'latitude', 'police_force', 'accident_severity', 'number_of_vehicles',
       'number_of_casualties', 'date', 'day_of_week', 'time',
       'local_authority_district', 'local_authority_ons_district',
       'local_authority_highway', 'first_road_class', 'first_road_number',
       'road_type', 'speed_limit', 'junction_detail', 'junction_control',
       'second_road_class', 'second_road_number',
       'pedestrian_crossing_human_control',
       'pedestrian_crossing_physical_facilities', 'light_conditions',
       'weather_conditions', 'road_surface_conditions',
       'special_conditions_at_site', 'carriageway_hazards',
       'urban_or_rural_area', 'did_police_officer_attend_scene_of_accident',
       'trunk_road_flag', 'lsoa_of_accident_location'],
      dtype='object')

In [ ]:
df_coll = df_collision[['accident_index', 'longitude', 'latitude',
                        'police_force', 'number_of_casualties',
                        'number_of_vehicles', 'did_police_officer_attend_scene_of_accident',
                        'road_type', 'junction_detail',
                        'accident_severity', 'date', 'day_of_week', 'time',
                        'weather_conditions','road_surface_conditions',
                        'light_conditions', 'speed_limit']]
df_coll.head()

,accident_index,longitude,latitude,police_force,number_of_casualties,number_of_vehicles,did_police_officer_attend_scene_of_accident,road_type,junction_detail,accident_severity,date,day_of_week,time,weather_conditions,road_surface_conditions,light_conditions,speed_limit
0,2018010080971,-0.139737,51.524587,1,2,2,1,3,0,3,01/01/2018,2,01:30,1,1,4,30
1,2018010080973,0.046471,51.539651,1,1,1,1,6,2,3,01/01/2018,2,00:50,1,1,4,30
2,2018010080974,-0.102474,51.529746,1,1,2,1,6,6,3,01/01/2018,2,00:45,1,1,4,20
3,2018010080981,0.037828,51.530179,1,1,2,1,3,7,2,01/01/2018,2,03:00,2,2,4,30
4,2018010080982,0.065781,51.469258,1,2,2,1,6,0,2,01/01/2018,2,02:20,1,2,4,30


In [ ]:
df_casaulty.columns

Index(['accident_index', 'accident_year', 'accident_reference',
       'vehicle_reference', 'casualty_reference', 'casualty_class',
       'sex_of_casualty', 'age_of_casualty', 'age_band_of_casualty',
       'casualty_severity', 'pedestrian_location', 'pedestrian_movement',
       'car_passenger', 'bus_or_coach_passenger',
       'pedestrian_road_maintenance_worker', 'casualty_type',
       'casualty_home_area_type', 'casualty_imd_decile', 'lsoa_of_casualty'],
      dtype='object')

In [ ]:
df_cas = df_casaulty[['accident_index', 'accident_year', 'sex_of_casualty',
                      'age_of_casualty', 'casualty_severity', 'casualty_type']]
df_cas.head()

,accident_index,accident_year,sex_of_casualty,age_of_casualty,casualty_severity,casualty_type
0,2018010080971,2018,2,50,3,9
1,2018010080971,2018,1,48,3,8
2,2018010080973,2018,1,29,3,0
3,2018010080974,2018,1,40,3,8
4,2018010080981,2018,1,27,2,9


In [ ]:
df_vehicle.columns

Index(['accident_index', 'accident_year', 'accident_reference',
       'vehicle_reference', 'vehicle_type', 'towing_and_articulation',
       'vehicle_manoeuvre', 'vehicle_direction_from', 'vehicle_direction_to',
       'vehicle_location_restricted_lane', 'junction_location',
       'skidding_and_overturning', 'hit_object_in_carriageway',
       'vehicle_leaving_carriageway', 'hit_object_off_carriageway',
       'first_point_of_impact', 'vehicle_left_hand_drive',
       'journey_purpose_of_driver', 'sex_of_driver', 'age_of_driver',
       'age_band_of_driver', 'engine_capacity_cc', 'propulsion_code',
       'age_of_vehicle', 'generic_make_model', 'driver_imd_decile',
       'driver_home_area_type', 'lsoa_of_driver'],
      dtype='object')

In [ ]:
df_veh = df_vehicle[['accident_index', 'sex_of_driver', 'age_of_driver',
                     'age_of_vehicle', 'generic_make_model',
                     ]]
df_veh.head()

,accident_index,sex_of_driver,age_of_driver,age_of_vehicle,generic_make_model
0,2018010080971,1,32,5,BMW 5 SERIES
1,2018010080971,1,48,6,TOYOTA PRIUS+
2,2018010080973,3,-1,-1,-1
3,2018010080974,1,40,6,TOYOTA PRIUS
4,2018010080974,1,21,-1,-1


### Table Joining

In [ ]:
#joining the city the highway is present
merged_df = pd.merge(df_coll, df_district, left_on='police_force', right_on='Index', how='left')
df_coll['police_dept'] = merged_df['County']
df_coll.head()

<ipython-input-15-a0c77ab6f4c8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coll['police_dept'] = merged_df['County']


,accident_index,longitude,latitude,police_force,number_of_casualties,number_of_vehicles,did_police_officer_attend_scene_of_accident,road_type,junction_detail,accident_severity,date,day_of_week,time,weather_conditions,road_surface_conditions,light_conditions,speed_limit,police_dept
0,2018010080971,-0.139737,51.524587,1,2,2,1,3,0,3,01/01/2018,2,01:30,1,1,4,30,Metropolitan Police
1,2018010080973,0.046471,51.539651,1,1,1,1,6,2,3,01/01/2018,2,00:50,1,1,4,30,Metropolitan Police
2,2018010080974,-0.102474,51.529746,1,1,2,1,6,6,3,01/01/2018,2,00:45,1,1,4,20,Metropolitan Police
3,2018010080981,0.037828,51.530179,1,1,2,1,3,7,2,01/01/2018,2,03:00,2,2,4,30,Metropolitan Police
4,2018010080982,0.065781,51.469258,1,2,2,1,6,0,2,01/01/2018,2,02:20,1,2,4,30,Metropolitan Police


In [ ]:
#merging all three data set together
df = (df_coll.merge(df_cas,  left_on = 'accident_index', right_on = 'accident_index')).merge(df_veh,  left_on = 'accident_index', right_on = 'accident_index')
df.shape

(945816, 27)

### Duplicates

In [ ]:
#checking for duplicates
df.duplicated().sum()
#result shows we have over 10,000 duplicates

13121

In [ ]:
#removing the duplicates
df = df.drop_duplicates()
df.shape

(932695, 27)

### Missing Values

In [ ]:
df.columns

Index(['accident_index', 'longitude', 'latitude', 'police_force',
       'number_of_casualties', 'number_of_vehicles',
       'did_police_officer_attend_scene_of_accident', 'road_type',
       'junction_detail', 'accident_severity', 'date', 'day_of_week', 'time',
       'weather_conditions', 'road_surface_conditions', 'light_conditions',
       'speed_limit', 'police_dept', 'accident_year', 'sex_of_casualty',
       'age_of_casualty', 'casualty_severity', 'casualty_type',
       'sex_of_driver', 'age_of_driver', 'age_of_vehicle',
       'generic_make_model'],
      dtype='object')

In [ ]:
#check for missing value
df.isna().any()

accident_index                                 False
longitude                                       True
latitude                                        True
police_force                                   False
number_of_casualties                           False
number_of_vehicles                             False
did_police_officer_attend_scene_of_accident    False
road_type                                      False
junction_detail                                False
accident_severity                              False
date                                           False
day_of_week                                    False
time                                           False
weather_conditions                             False
road_surface_conditions                        False
light_conditions                               False
speed_limit                                    False
police_dept                                    False
accident_year                                 

In [ ]:
df[['longitude','latitude']].isna().sum()



longitude    283
latitude     283
dtype: int64

In [ ]:
#dropping null values in longitude and latitude column
df= df.dropna(subset= ['longitude', 'latitude'])
df.shape

(932412, 27)

In [ ]:
#missing data were coded with -1, converting them to null to deal with them properly
for i in df.columns:
  df[i]= df[i].replace({
      '-1': np.nan,
      -1 : np.nan
  })

df.isna().any()

accident_index                                 False
longitude                                      False
latitude                                       False
police_force                                   False
number_of_casualties                           False
number_of_vehicles                             False
did_police_officer_attend_scene_of_accident     True
road_type                                      False
junction_detail                                 True
accident_severity                              False
date                                           False
day_of_week                                    False
time                                           False
weather_conditions                              True
road_surface_conditions                         True
light_conditions                                True
speed_limit                                     True
police_dept                                    False
accident_year                                 

In [ ]:
#replacing the null values for categorical variables
#did_police_officer_attend_scene_of_accident, null values were replaced with 3
df['did_police_officer_attend_scene_of_accident']= df['did_police_officer_attend_scene_of_accident'].replace({np.nan: 3})
#junction_details, null values replaced with the mode of the distribution
df['junction_detail']= df['junction_detail'].replace({np.nan:(df['junction_detail'].mode()[0])})
#null replaced with last valid observation for the following variable
df['weather_conditions'].fillna(method = 'bfill', inplace = True)
df['road_surface_conditions'].fillna(method = 'bfill', inplace = True)
df['light_conditions'].fillna(method = 'bfill', inplace = True)
df['speed_limit'].fillna(method = 'bfill', inplace = True)
df['casualty_type'].fillna(method = 'bfill', inplace = True)
#generic_make_model, null values were replaced with unknown
df['generic_make_model']= df['generic_make_model'].replace({np.nan: 'unkown'})
#sex_of_casualty, null values replaced with 9, unknown.
df['sex_of_driver']= df['sex_of_driver'].replace({np.nan: 3})
df['sex_of_casualty']= df['sex_of_casualty'].replace({np.nan: 3})

In [ ]:
#replacing the null values for numerical variables
#speed_limit, #null replaced with last valid observation for data consistency
df['speed_limit'].fillna(method = 'bfill', inplace = True)
#KNN imputer used to fill the null value for other numeric variables
#setting the hyperparameters of the imputer by setting the number of
#neighbours to 5
imputer = KNNImputer(n_neighbors=5)

#fitting the KNNimputer on the trainset and transforming the trainset
#using the imputer
df[['number_of_casualties','number_of_vehicles',
    'age_of_casualty','age_of_driver',
    'age_of_vehicle']] = imputer.fit_transform(df[['number_of_casualties',
                                                   'number_of_vehicles',
                                                   'age_of_casualty',
                                                   'age_of_driver',
                                                   'age_of_vehicle']])

In [ ]:
df[['number_of_casualties','number_of_vehicles',
    'age_of_casualty','age_of_driver',
    'age_of_vehicle']] = df[['number_of_casualties','number_of_vehicles',
    'age_of_casualty','age_of_driver',
    'age_of_vehicle']].apply(np.ceil)

In [ ]:
df.isna().any()

accident_index                                 False
longitude                                      False
latitude                                       False
police_force                                   False
number_of_casualties                           False
number_of_vehicles                             False
did_police_officer_attend_scene_of_accident    False
road_type                                      False
junction_detail                                False
accident_severity                              False
date                                           False
day_of_week                                    False
time                                           False
weather_conditions                             False
road_surface_conditions                        False
light_conditions                               False
speed_limit                                    False
police_dept                                    False
accident_year                                 

### Data Type

In [ ]:
#checking for the data type of each variable
df.dtypes

accident_index                                  object
longitude                                      float64
latitude                                       float64
police_force                                     int64
number_of_casualties                           float64
number_of_vehicles                             float64
did_police_officer_attend_scene_of_accident    float64
road_type                                        int64
junction_detail                                float64
accident_severity                                int64
date                                            object
day_of_week                                      int64
time                                            object
weather_conditions                             float64
road_surface_conditions                        float64
light_conditions                               float64
speed_limit                                    float64
police_dept                                     object
accident_y

In [ ]:
#setting the datatype for the wrong ones
df["date"] = pd.to_datetime(df['date'], format = '%d/%m/%Y')
df["time"] = pd.to_datetime(df['time'], format = '%H:%M')
df["accident_index"] = df["accident_index"].astype("string")
df["longitude"] = df["longitude"].astype("string")
df["latitude"] = df["latitude"].astype("string")
df["accident_year"] = df["accident_year"].astype("category")
df["police_force"] = df["police_force"].astype("category")
df["police_dept"] = df["police_dept"].astype("category")
df["did_police_officer_attend_scene_of_accident"] = df["did_police_officer_attend_scene_of_accident"].astype("category")
df["road_type"] = df["road_type"].astype("category")
df["day_of_week"] = df["day_of_week"].astype("category")
df["junction_detail"] = df["junction_detail"].astype("category")
df["accident_severity"] = df["accident_severity"].astype("category")
df["weather_conditions"] = df["weather_conditions"].astype("category")
df["road_surface_conditions"] = df["road_surface_conditions"].astype("category")
df["light_conditions"] = df["light_conditions"].astype("category")
df["sex_of_casualty"] = df["sex_of_casualty"].astype("category")
df["casualty_severity"] = df["casualty_severity"].astype("category")
df["casualty_type"] = df["casualty_type"].astype("category")
df["sex_of_driver"] = df["sex_of_driver"].astype("category")
df["generic_make_model"] = df["generic_make_model"].astype("category")


In [ ]:
df.dtypes

accident_index                                 string[python]
longitude                                      string[python]
latitude                                       string[python]
police_force                                         category
number_of_casualties                                  float64
number_of_vehicles                                    float64
did_police_officer_attend_scene_of_accident          category
road_type                                            category
junction_detail                                      category
accident_severity                                    category
date                                           datetime64[ns]
day_of_week                                          category
time                                           datetime64[ns]
weather_conditions                                   category
road_surface_conditions                              category
light_conditions                                     category
speed_li

### Identifying  Outliers using Zscore

Using the following formula to identify the z score:

zscore= (observation - mean)/ standard deviation

z = (X - x̄)/σ

Outliers have  an absolute z score value greater than 3 (threshold)

In [ ]:
#Create a function to check for outliers
#outliers can be identified on numeric variables
#we are using a function because we have more than one variable to check
#using z score
def find_outlier(x):
  thresh = 3
  mean = x.mean()
  std = x.std()
  z_score = (x - mean)/std
  return x[np.abs(z_score) > thresh]

In [ ]:
casualty_age_out = find_outlier(df['age_of_casualty'])
casualty_age_out

In [ ]:
vehicle_age_out = find_outlier(df['age_of_vehicle'])
vehicle_age_out

In [ ]:
vehicle_numbers_out = find_outlier(df['number_of_vehicles'])
vehicle_numbers_out

In [ ]:
casualty_number_out = find_outlier(df['number_of_casualties'])
casualty_number_out

In [ ]:
drive_age_out = find_outlier(df['age_of_driver'])
drive_age_out

There were outliers present in all variables. This will be confirmed in the exploratory data analysis and will be dealt with before proceeding to predictive analysis

### Feature Engineering

In [ ]:
#feature engineering
#using latitude and longitude of accident to get the county of the accident
#initialize Nominatim geocoder
geolocator = Nominatim(user_agent="geoapiExercises")
def get_city(row):
    location = geolocator.reverse((row['latitude'], row['longitude']), exactly_one=True)
    return location.raw['address'].get('city', 'Unknown')

# Apply the function to create a new 'City' column
df['city'] = df.apply(get_city, axis=1)

#setting the datatype
df["city"] = df["city"].astype("category")


GeocoderInsufficientPrivileges: Non-successful status code 403

In [ ]:
#getting the time of the day
bins = [0, 6, 12, 18, 24]
labels = ['Night', 'Morning', 'Afternoon', 'Evening']

#encode the data using labels, I forgot to do this.

# Create a new column 'time_category' based on time intervals
df['time_category'] = pd.cut(df['time'].dt.hour, bins=bins, labels=labels, right=False)

#setting the datatype
df["time_category"] = df["time_category"].astype("category")

In [ ]:
#getting the season of the year
#getting months from date
df['month'] = df['date'].dt.month
#dictionary holding months and seasons
seasons = {
    1: '1',
    2: '1',
    3: '2',
    4: '2',
    5: '2',
    6: '3',
    7: '3',
    8: '3',
    9: '4',
    10: '4',
    11: '4',
    12: '1'
}

#map month to season
df['season'] = df['month'].map(seasons)

#setting the datatype
df["season"] = df["season"].astype("category")
df["month"] = df["month"].astype("category")

In [ ]:
df.head()

,accident_index,longitude,latitude,police_force,number_of_casualties,number_of_vehicles,did_police_officer_attend_scene_of_accident,road_type,junction_detail,accident_severity,...,age_of_casualty,casualty_severity,casualty_type,sex_of_driver,age_of_driver,age_of_vehicle,generic_make_model,time_category,month,season
0,2018010135259,-0.07312,51.398512,1,1.0,2.0,2.0,6,3.0,3,...,18.0,3,9.0,2.0,18.0,0.0,MINI ONE,Afternoon,9,4
1,2018010135259,-0.07312,51.398512,1,1.0,2.0,2.0,6,3.0,3,...,18.0,3,9.0,3.0,26.0,16.0,MAZDA 6,Afternoon,9,4
2,2018010135261,-0.043058,51.550261,1,1.0,1.0,2.0,6,3.0,3,...,30.0,3,0.0,3.0,30.0,9.0,unkown,Morning,9,4
3,2018010135264,-0.064263,51.61482,1,1.0,1.0,1.0,2,9.0,3,...,35.0,3,0.0,1.0,56.0,1.0,AUDI A7,Morning,9,4
4,2018010135268,-0.147913,51.347739,1,2.0,2.0,1.0,1,2.0,3,...,73.0,3,9.0,1.0,78.0,8.0,HONDA JAZZ,Evening,9,4


In [ ]:
_ = load_dontenv(find_dotenv()) #read local .env file

apikey = os.environ['GoogleApiKey']
apikey

In [ ]:
#getting distance to setting location
#got this segment from chatgpt and refined it to my dataset.
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371 * c  # Radius of the Earth in kilometers
    return distance

#distance to the location
#I used google map api
def get_distance_to_location(latitude, longitude, radius=5000,typical):
    #use your google maps api key
    api_key = apikey
    url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={latitude},{longitude}&radius={radius}&type={typical}&key={api_key}'
    response = requests.get(url)
    data = response.json()
    locations = []
    if 'results' in data:
        for result in data['results']:
            locations.append({
                'name': result['name'],
                'latitude': result['geometry']['location']['lat'],
                'longitude': result['geometry']['location']['lng']
            })
    return locations



In [ ]:
#get distance to nearby hospital from accident
def calculate_distance_to_nearest_hospital(df, lat_column, lon_column):
    distances = []
    for index, row in df.iterrows():
        accident_latitude = row[lat_column]
        accident_longitude = row[lon_column]
        typical = 'hospital'
        hospitals = get_distance_to_location(accident_latitude, accident_longitude,typical)
        nearest_distance = float('inf')
        for hospital in hospitals:
            distance = haversine(accident_latitude, accident_longitude, hospital['latitude'], hospital['longitude'])
            if distance < nearest_distance:
                nearest_distance = distance
        distances.append(nearest_distance)
    df['distance_to_hospital'] = distances
    return df
  df = calculate_distance_to_nearest_hospital(df, 'latitude', 'longitude')
  df["distance_to_hospital"] = df["distance_to_hospital"].astype("float")

In [ ]:
#get distance to the nearest town centre
def calculate_distance_to_nearest_town_center(df, lat_column, lon_column):
    distances = []
    for index, row in df.iterrows():
        accident_latitude = row[lat_column]
        accident_longitude = row[lon_column]
        typical = 'locality'
        town_centers = get_nearby_town_centers(accident_latitude, accident_longitude, typical)
        nearest_distance = float('inf')
        for town_center in town_centers:
            distance = haversine(accident_latitude, accident_longitude, town_center['latitude'], town_center['longitude'])
            if distance < nearest_distance:
                nearest_distance = distance
        distances.append(nearest_distance)
    df['distance_to_town_center'] = distances
    return df
df = calculate_distance_to_nearest_town_center(df, 'latitude', 'longitude')
df["distance_to_town_center"] = df["distance_to_town_center"].astype("float")

### Saving the new data set

In [ ]:
#saving the clean dataset to be accessible anytime
df.to_csv('/content/drive/MyDrive/cleaned-uk-road-data-last-5-years.csv')